In [2]:
from IPython.display import Markdown

In [1]:
import openai
from dotenv import load_dotenv
from openai import OpenAI
import os
from langchain.prompts import PromptTemplate

env_loaded = load_dotenv()
print("Environment loaded:", env_loaded)



client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)


Environment loaded: True


In [3]:
def get_response(prompt_question):
    response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful AI assistant"},
                {"role": "user", "content": prompt_question}
            ],
            )
    return response.choices[0].message.content



In [4]:
Markdown(get_response("What is a 3 body problem?"))

The three-body problem is a classical problem in physics and astronomy that involves predicting the motion of three celestial bodies interacting with each other through gravitational forces. Unlike the simpler two-body problem, which involves only two celestial bodies and can be solved exactly using Kepler's laws of celestial mechanics, the three-body problem is significantly more complex and typically cannot be solved exactly. Instead, solutions to the three-body problem often require numerical methods and simulations.

Key characteristics of the three-body problem include:
1. **Complex Dynamics**: The gravitational interactions among the three bodies lead to highly complex and often chaotic trajectories, making analytical solutions difficult.
2. **Initial Conditions**: The specific initial positions and velocities of the three bodies play a crucial role in determining their future motion.
3. **Nonlinear Equations**: The differential equations governing the motion of the three bodies are nonlinear, which contributes to the complexity of finding solutions.

Historically, the study of the three-body problem has led to significant advances in mathematics and physics, including the development of chaos theory and perturbation theory. In practice, solving the three-body problem often involves using computer algorithms to simulate the interactions over time.

Some special cases of the three-body problem, where the bodies follow specific constrained paths or exhibit certain symmetries, have analytical solutions. However, for general initial conditions and configurations, numerical methods are typically required to study and understand the problem.

In [5]:
chat= [
    {
      "user": "Alice",
      "message": "Hey Bob, how's it going?"
    },
    {
      "user": "Bob",
      "message": "Hi Alice! I'm good, how about you?"
    },
    {
      "user": "Alice",
      "message": "I'm great, thanks! Do you want to catch up this weekend?"
    },
    {
      "user": "Bob",
      "message": "Sure, that sounds good. How about Saturday afternoon?"
    },
    {
      "user": "Alice",
      "message": "Saturday works for me. Let's meet at 3 PM at the usual café."
    },
    {
      "user": "Bob",
      "message": "Perfect, I'll see you then!"
    },
    {
      "user": "Alice",
      "message": "Great, looking forward to it! See you on Saturday at 3 PM."
    }
  ]

In [9]:
def get_conv(chat):
    
    chat_string = ''
    for message in chat:            
        chat_string += f"{message['user']}: {message['message']} \n"
    
    return chat_string

In [10]:
get_conv(chat)

"Alice: Hey Bob, how's it going? \nBob: Hi Alice! I'm good, how about you? \nAlice: I'm great, thanks! Do you want to catch up this weekend? \nBob: Sure, that sounds good. How about Saturday afternoon? \nAlice: Saturday works for me. Let's meet at 3 PM at the usual café. \nBob: Perfect, I'll see you then! \nAlice: Great, looking forward to it! See you on Saturday at 3 PM. \n"

In [11]:
context = """ 

You are an AI agent that takes necessary actions if needed based on the conversation provided. You have a set of tools available, pick the available tool wisely.

tools: 

def create_event(date, *kwargs):
    #Creates an event in the user's calander via API call.
    
def set_reminder(date, task):
    #Sets an immediate reminder for the tasks
    

conversation: {chat}
YOU ONLY OUTPUT THE FUNCTION TO CALL
"""

In [12]:
answer_prompt = PromptTemplate(template=context, input_variables=['chat'])
formatted_prompt = answer_prompt.format(chat=get_conv(chat) )

In [13]:
formatted_prompt

" \n\nYou are an AI agent that takes necessary actions if needed based on the conversation provided. You have a set of tools available, pick the available tool wisely.\n\ntools: \n\ndef create_event(date, *kwargs):\n    #Creates an event in the user's calander via API call.\n    \ndef set_reminder(date, task):\n    #Sets an immediate reminder for the tasks\n    \n\nconversation: Alice: Hey Bob, how's it going? \nBob: Hi Alice! I'm good, how about you? \nAlice: I'm great, thanks! Do you want to catch up this weekend? \nBob: Sure, that sounds good. How about Saturday afternoon? \nAlice: Saturday works for me. Let's meet at 3 PM at the usual café. \nBob: Perfect, I'll see you then! \nAlice: Great, looking forward to it! See you on Saturday at 3 PM. \n\nYOU ONLY OUTPUT THE FUNCTION TO CALL\n"

In [14]:
Markdown(get_response(formatted_prompt))

```python
create_event('Saturday 3 PM', name='Catch up with Bob', location='usual café')
```

In [15]:
Markdown(get_response("What's the time?/"))

I'm unable to check the current time as I don't have access to real-time information. However, you can easily check the time on your device or by searching for the current time in your location online. How else may I assist you today?

In [49]:
from datetime import datetime, timedelta

def get_current_time():
    """
    Gets the current time in datetime format along with the current day of the week
    """
    now = datetime.now()
    current_time = now.strftime("%Y-%m-%d %H:%M:%S")
    current_day = now.strftime("%A")  # %A gives the full weekday name, e.g., Monday
    return f"{current_day}, {current_time}"


In [50]:
get_current_time()

'Saturday, 2024-06-01 20:22:12'

In [54]:
def create_event(start_time, description, end_time=30, location=None, summary= None):

    """ Creates a google calander event given the start_time and description as args.
        Also takes in kwargs like end_time, location, summary
        end_time is by default set to 30mins after start time. 
        location and summary defaults to None
    """

    # Parse the start time to a datetime object
    start_datetime = datetime.strptime(start_time, "%Y-%m-%dT%H:%M:%S")
    
    # Calculate the end time by adding the duration to the start time
    end_datetime = start_datetime + timedelta(minutes=end_time)

    # Create the event body
    event = {
        'description': description,
        'start': {
            'dateTime': start_datetime.isoformat(),
            'timeZone': 'UTC',  # Adjust time zone as needed
        },
        'end': {
            'dateTime': end_datetime.isoformat(),
            'timeZone': 'UTC',  # Adjust time zone as needed
        },
        'reminders': {
            'useDefault': False,
            'overrides': [
                {'method': 'email', 'minutes': 24 * 60},
                {'method': 'popup', 'minutes': 10},
            ],
        },
    }

    # Add location if provided
    if location:
        event['location'] = location
    
    if summary:
        event['summary'] = summary
    else:
        event['summary'] = 'New event'
    print(event)

    if start_datetime < datetime.now():
        return "Start time can not be in past"
    else:
        return "Event Scheduled"
# Example usage
create_event(
    start_time='2024-06-01T15:00:00',
    description='Discussing the upcoming project.',
    end_time=30,
    location='123 Main St, Anytown, USA'
)

{'description': 'Discussing the upcoming project.', 'start': {'dateTime': '2024-06-01T15:00:00', 'timeZone': 'UTC'}, 'end': {'dateTime': '2024-06-01T15:30:00', 'timeZone': 'UTC'}, 'reminders': {'useDefault': False, 'overrides': [{'method': 'email', 'minutes': 1440}, {'method': 'popup', 'minutes': 10}]}, 'location': '123 Main St, Anytown, USA', 'summary': 'New event'}


TypeError: '<' not supported between instances of 'str' and 'datetime.datetime'

In [51]:


def run_conversation(query):
    
    
    
    messages=[
                {"role": "system", "content": "You are an AI agent"},
                {"role": "user", "content": query}
            ]
    
    tools = [
        
        {
            "type": "function",
            "function": {
                "name": "get_current_time",
                "description": "Gets the current day and time in datetime format which might be required for event scheduling",
                
                
            }
        },
        
        {
            "type": "function",
            "function": {
                
                "name": "create_event",
                "description": "Creates a google calander event given the start_time and description as args. Also takes in kwargs like end_time, location, summary. end_time is by default set to 30. location and summary defaults to None",
                "parameters": {
                    
                    "type": "object",
                    "properties": {
                        
                        "start_time": {
                            
                            "type": "string",
                            "description": "Start time of the event, in a datetime format string, you need current time to adjust this event ",
                        },
                        
                        "description": {
                            
                            "type": "string",
                            "description": "Description of the event",
                        },
                        
                        "end_time": {
                            
                            "type": "integer",
                            "description": "duration of the event, by default set to 30",
                        },
                        
                        "location": {
                            
                            "type": "string",
                            "description": "location of the event, by default set to None",
                        },
                        
                        "summary": {
                            
                            "type": "string",
                            "description": "summary of the event, by default set to None"
                        }
                        
                    }
                }
            }
            
        }    
    ]
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    
    return response

In [52]:
response = run_conversation(formatted_prompt)

In [53]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_922XtT9w289aF91gUXAT3F5s', function=Function(arguments='{"start_time":"2023-10-07T15:00:00","description":"Catch up with Alice at the usual café.","summary":"Meet with Alice"}', name='create_event'), type='function')]

In [47]:
response_message

NameError: name 'response_message' is not defined